# Tutorial
Note, this tutorial corresponds to the jupyter notebook found at `xmen/examples/tutorial.ipynb`. After installing xmen you should be able to run this notebook on your local machine.

## Defining Experiments
Experiments can be defined in one of two ways:
1. As python functions
2. As python classes

### Functional Experiments
In the functional API experiments are defined as functions:
```python
from xmen import Root

def hello_world(
    root: Root,   # experiments are assigned a root before being executed
    a: str = 'Hello',  # the first
    # argument comment spans multiple lines
    b: str = 'World'   # the second argument
):
    """A hello world experiment designed to demonstrate
    defining experiments through the functional experiment api"""
    print(f'{a}  {b}')

    ...  #  Whatever other experiment code you want

    with open(root.directory + '/out.txt', 'w') as f:
        f.write(f'{a} {b}')
    root.message({'a': a, 'b': b})
```
Arguments to the function are treated as parameters, with comments next to each parameter treated as documentation. You are also encouraged to specify parameters with their type and a default value, alongside specifying the doc string of the experiment.

Before being run each experiment is first linked to a driectory. This is supplied to the experiment as the first argument - defined as `root` in the example above. Principally, two methods are exposed to the experiment through root:
1. `root.directory` gives the directory the experiment is linked to
2. `root.message()` allows the experiment to leave simple messages (either floats, ints or strings) with the experiment root


### Class Experiments
Alternatively experiments can be specified as python classes:

```python
from xmen import Experiment

class HelloWorld(Experiment):
    """A hello world experiment designed to demonstrate
    defining experiments through the class experiment api"""
    # Parameters
    a: str = 'Hello'  # @p The first argument
    b: str = 'World'  # @p The second argument

    def run(self):
        print(f'{self.a} {self.b}!')
        
        with open(self.directory + '/out.txt', 'w') as f:
            f.write(f'{a} {b}')
        self.message({'a': self.a, 'b': self.b})
```
In this case, all experiments must inherit from ``Experiment``. Parameters are defined as class attributes marked with a `# @p` comment - any method not marked with ``# @p`` will not be regarded as a parameter. The directory and messaging method are directly available to the experiment as class attributes.

Defining experiment as python classes allows experiments to benefit from all the features of an object orientated programming approach including encapsulation and inheritance. Further examples showing the benefits of defining experiments as classes can be found in:
- ``/xmen/examples/inheritance.py``
- ``/xmen/examples/torch/inheritance.py``

and is documented further in the ``class-api.ipynb``. 

## Xmen command line interface
Any experiment that conforms to the xmen api can automatically be run in the commandline. This is facilitated by the `xmen` command line tool:

In [1]:
%%bash
xmen

usage: xmen [-h] [--list] [--add MODULE NAME MODULE NAME] [--remove REMOVE]
            [name [name ...]] ...

||||||||||||||||||||||||| WELCOME TO ||||||||||||||||||||||||||
||                                                           ||
||    \\\  ///  |||\\        //|||  |||||||||  |||\\   |||   ||
||     \\\///   |||\\\      ///|||  |||        |||\\\  |||   ||
||      ||||    ||| \\\    /// |||  ||||||     ||| \\\ |||   ||
||     ///\\\   |||  \\\  ///  |||  |||        |||  \\\|||   ||
||    ///  \\\  |||   \\\///   |||  |||||||||  |||   \\|||   ||
||                                                           ||
|||||||||||| FAST - REPRODUCIBLE - EXPERIMENTATION ||||||||||||

positional arguments:
  name                  The name of the experiment to run
  flags                 Python flags (pass --help for more info)

optional arguments:
  -h, --help            show this help message and exit
  --list, -l            List available python experiments
  --add MODULE NAME MODULE NAME


### Adding an Experiment to Xmen

Before being run an experiment must first be registered with ``xmen``. This is achieved by passing the module and the name of each experiment. For example the experiments defined above which are copies of ``hello_world`` and ``HelloWorld`` in `xmen.examples.hello_world` can be added as:

In [2]:
%%bash
xmen --add xmen.examples.hello_world hello_world
xmen --add xmen.examples.hello_world HelloWorld

They can now be accessed in ``xmen``:

In [3]:
%%bash 
xmen hello_world

A hello world experiment designed to demonstrate
    defining experiments through the functional experiment api

Parameters:
    a: str=Hello ~ the first argument
    b: str=World ~ the second argument

For more help use --help.


Xmen automatically takes care of the command line interface including automatically reading comments next to parameters and adding them to the experiments help, alonsidge the experiments `__doc__` string.

### Executing Experiments
Experiments are executed by passing the `-x` command line argument alongside a path to the experiments repository:

In [4]:
%%bash
xmen hello_world -x ~/tmp/hello_world

Hello  World


The parameters of the experiment can be updated by using the `-u` command line flag, specifying the parameters to update as a yaml dictionary. For example:

In [5]:
%%bash
xmen hello_world -u "{a: Bye Bye, b: planet}" -x ~/tmp/bye_bye

Updating parameters {'a': 'Bye Bye', 'b': 'planet'}
Bye Bye  planet


### Automatic Record Keeping
Alongside running the experiment `xmen` also logs all the important information to reproduce the experiment at a later date. This is logged in a ``params.yml`` in the experiment repository. For example:

In [6]:
%%bash
cat ~/tmp/bye_bye/params.yml

_created: 11-16-20-21:31:30  # _created: str=now_time ~ The date the experiment was created
_messages: # _messages: Dict[Any, Any]={} ~ Messages left by the experiment
  a: Bye Bye
  b: planet
_meta: # _meta: Optional[Dict]=None ~ The global configuration for the experiment manager
  mac: '0x6c96cfdb71b9'
  host: robw-macbook-2.local
  user: robweston
  home: /Users/robweston
_name: bye_bye # _name: Optional[str]=None ~ The name of the experiment (under root)
_purpose: '' # _purpose: Optional[str]=None ~ A description of the experiment purpose
_root: /Users/robweston/tmp # _root: Optional[str]=None ~ The root directory of the experiment
_status: finished # _status: str='default' ~ One of ['default' | 'created' | 'running' | 'error' | 'finished']
_version: # _version: Optional[Dict[Any, Any]]=None ~ Experiment version information. See `get_version`
  module: xmen.examples.hello_world
  function: hello_world
  path: /Users/robweston/xmen/xmen/examples/hello_world.py
  git:
    local: /Us

Here we can find:
- `_created` date and time the experiment was run
- `_messages` any messages logged with the experiment during execution
- `_meta` information about the system the experiment was run on
- `_status` the state of the experiment. If an error occurs during excution then the experiment state will be updated to `'errror'`.
- `_version` the information needed to re-run the experiment. If the file lives in a git repo then the git commit, path to the repositoy and remote repository are also loggeed automatically.


## Xgent command line interface

The e`x`periment mana`g`em`ent` tool `xgent` allows multiple experiments to be quickly configured and run from the command line and is designed to easily interface with the slurm job scheduler.

In [7]:
%%bash
xgent

usage: xgent [-h]
             {config,init,register,run,note,reset,unlink,clean,rm,relink,list}
             ...

||||||||||||||||||||||||||| WELCOME TO ||||||||||||||||||||||||||||
||                                                              ||
||          &@&@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@&@&%          ||    
||         *@&@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@&         ||    
||          &@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@&          ||    
||           &@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@&          ||    
||           &@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@#          ||    
||           &@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@&.          ||    
||           &@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@.          ||    
||           &@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@*          ||    
||           @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@          ||    
||   #&@@@@@&%&&&&&&&@@@@@@@@@@@@@@@@@@@@@@@@@&&&&&&&&@@@@@@&#  ||    
||  /#%%%%%%%%%&&@@@@@@@@@@@@@@@@@@@@@@@

### Initialising a set of experiments
A set of experiments is initialised from a python experiment definition using the `xgent init` command. For example,

In [8]:
%%bash
xgent init -n hello_world -r ~/tmp/xgent/hello_world

Generating experiment root at /Users/robweston/tmp/xgent/hello_world
hello_world
Experiment root created at /Users/robweston/tmp/xgent/hello_world


### Update Defaults

After a set of experiments has been initialised, the default parameters of the experiment can be changed by changing the `defaults.yml` generated by xgent suring the `xgent init` call:

In [9]:
cd ~/tmp/xgent/hello_world

/Users/robweston/tmp/xgent/hello_world


In [10]:
%%bash
cat defaults.yml

_created: 11-16-20-23:19:21  # _created: str=now_time ~ The date the experiment was created
_meta: # _meta: Optional[Dict]=None ~ The global configuration for the experiment manager
  mac: '0x6c96cfdb71b9'
  host: robw-macbook-2.local
  user: robweston
  home: /Users/robweston
_version: # _version: Optional[Dict[Any, Any]]=None ~ Experiment version information. See `get_version`
  module: xmen.examples.hello_world
  function: hello_world
  path: /Users/robweston/xmen/xmen/examples/hello_world.py
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: c434b15c09937e5dbd4b0e3391943c272df884da
    branch: master
a: Hello # a: str=Hello ~ the first argument
b: World # b: str=World ~ the second argument


### Register an experiment

After this experiments are registered to the experiment set using the ``xgent register`` command:

In [11]:
%%bash
xgent register -u "{a: Hello | Bye, b: World | Planet}"

The `-u` flag is used to specify the parameters that should be updated for each parameter instance. It is passed as a yaml dictionary with the `|` character used as an or operator. For example, in this case four experiments are registered corresponding to:

| a       | b      |
|---------|--------|
| Hello   | World  |
| Hello   | Planet |
| Bye     | World  |
| Bye     | Planet |


The experiments registered in the experiment set are summarised using the `xgent list` command:

In [12]:
%%bash
xgent list -sd

/Users/robweston/tmp/xgent/hello_world*
          root               name      status              created
0  hello_world   a=Hello__b=World  registered  2020-11-16-23-19-26
1  hello_world  a=Hello__b=Planet  registered  2020-11-16-23-19-26
2  hello_world     a=Bye__b=World  registered  2020-11-16-23-19-26
3  hello_world    a=Bye__b=Planet  registered  2020-11-16-23-19-26

Roots relative to: /Users/robweston/tmp/xgent


### Run batches of experiments

After registering experiments are run using the `xgent run` command:

In [13]:
%%bash
xgent run "*" bash

Bye  Planet
Bye  World
Hello  Planet
Hello  World

Running: bash /Users/robweston/tmp/xgent/hello_world/a=Bye__b=Planet/run.sh

Running: bash /Users/robweston/tmp/xgent/hello_world/a=Bye__b=World/run.sh

Running: bash /Users/robweston/tmp/xgent/hello_world/a=Hello__b=Planet/run.sh

Running: bash /Users/robweston/tmp/xgent/hello_world/a=Hello__b=World/run.sh


If we now run

In [14]:
%%bash
xgent list -sd -m "a|b"

/Users/robweston/tmp/xgent/hello_world*
          root               name    status              created      a       b
0  hello_world   a=Hello__b=World  finished  2020-11-16-23-19-32  Hello   World
1  hello_world  a=Hello__b=Planet  finished  2020-11-16-23-19-32  Hello  Planet
2  hello_world     a=Bye__b=World  finished  2020-11-16-23-19-31    Bye   World
3  hello_world    a=Bye__b=Planet  finished  2020-11-16-23-19-31    Bye  Planet

Roots relative to: /Users/robweston/tmp/xgent


the experiment status has been changed from registered to finished and each message has been logged. Once finished experiments cannot be re-run unless they are first re-registered. This is achieved using the `xgent reset` command:

In [15]:
%%bash
xgent reset "*"
xgent run "*" bash
xgent list -sd -m "a|b"

Bye  Planet
Bye  World
Hello  Planet
Hello  World

Running: bash /Users/robweston/tmp/xgent/hello_world/a=Bye__b=Planet/run.sh

Running: bash /Users/robweston/tmp/xgent/hello_world/a=Bye__b=World/run.sh

Running: bash /Users/robweston/tmp/xgent/hello_world/a=Hello__b=Planet/run.sh

Running: bash /Users/robweston/tmp/xgent/hello_world/a=Hello__b=World/run.sh
/Users/robweston/tmp/xgent/hello_world*
          root               name    status              created      a       b
0  hello_world   a=Hello__b=World  finished  2020-11-16-23-19-39  Hello   World
1  hello_world  a=Hello__b=Planet  finished  2020-11-16-23-19-38  Hello  Planet
2  hello_world     a=Bye__b=World  finished  2020-11-16-23-19-38    Bye   World
3  hello_world    a=Bye__b=Planet  finished  2020-11-16-23-19-37    Bye  Planet

Roots relative to: /Users/robweston/tmp/xgent


In this case notice how the created time has been updated. This correpsonds to the time the experiment was last run. Various options exist for running experiments using the `xgent run` command. For example we could run each experiment in a screen command as:

In [16]:
%%bash
xgent reset "*"
xgent run "*" screen -dm bash
xgent list -sd -m "a|b"


Running: screen -dm bash /Users/robweston/tmp/xgent/hello_world/a=Bye__b=Planet/run.sh

Running: screen -dm bash /Users/robweston/tmp/xgent/hello_world/a=Bye__b=World/run.sh

Running: screen -dm bash /Users/robweston/tmp/xgent/hello_world/a=Hello__b=Planet/run.sh

Running: screen -dm bash /Users/robweston/tmp/xgent/hello_world/a=Hello__b=World/run.sh
/Users/robweston/tmp/xgent/hello_world*
          root               name    status              created      a       b
0  hello_world   a=Hello__b=World  finished  2020-11-16-23-19-45  Hello   World
1  hello_world  a=Hello__b=Planet  finished  2020-11-16-23-19-45  Hello  Planet
2  hello_world     a=Bye__b=World  finished  2020-11-16-23-19-44    Bye   World
3  hello_world    a=Bye__b=Planet  finished  2020-11-16-23-19-44    Bye  Planet

Roots relative to: /Users/robweston/tmp/xgent


### Interfacing with the slurm job scheduler

Another powerful option for running experiments on high performance computing clusters is the slurm job scheduler. To run experiments using the slurm job scheduler first add a header,

In [17]:
%%bash
cat ~/xmen/examples/header.txt

#SBATCH --nodes=1
#SBATCH --job-name=single_job
#SBATCH --time=1-00:00:00
#SBATCH --gres=gpu:1
#SBATCH --constraint='gpu_sku:RTX|gpu_sku:V100-LS|gpu_sku:V100|gpu_sku:P100'
#SBATCH --partition=htc-nova,htc
#SBATCH --cpus-per-task=2
#SBATCH --mail-user=robw@robots.ox.ac.uk
#SBATCH --mail-type=ALL
#SBATCH --account=engs-a2i
#SBATCH --signal=SIGUSR1@90
# Author: Rob Weston
# Email: robw@robots.ox.ac.uk


to the global configuration:

In [18]:
%%bash
xgent config -H ~/xmen/examples/header.txt

This will be appended to each run script generated by xgent. Experiments can then be submitted using the steps described before, but this time running each job with the sbatch command:

```python
xgent run "*" sbatch
```

## Next Steps
- For further help running any of the command line tools try `xmen --help`, `xmen hello_world --help`, `xgent --help` and `xgent ... --help`
- More exampels of the experiments can be found in `xmen.examples`
- Check out the API docs and the cheat sheet